Look, The core training hyparams, precision,optimization,logging etc depends on your data, how much data you have and what kind of , Here for a 2k Example -

In [ ]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(2_000))
N           = 2_000           # now only 2k samples

batch_size  = 1
grad_accum  = 8
num_epochs  = 3
steps_per_epoch = math.ceil(N / (batch_size * grad_accum))
total_steps     = num_epochs * steps_per_epoch
warmup_steps    = int(0.03 * total_steps)

trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = dataset["train"],
    #formatting_func     = formatting_func,
    max_seq_length      = 512,
    packing             = False,
    args = TrainingArguments(
        per_device_train_batch_size   = batch_size,
        gradient_accumulation_steps   = grad_accum,
        warmup_steps                  = warmup_steps,
        max_steps                     = total_steps,
        learning_rate                 = 1e-4,
        fp16                          = not is_bfloat16_supported(),
        bf16                          = is_bfloat16_supported(),
        logging_steps                 = 10,
        optim                         = "adamw_8bit",
        weight_decay                  = 0.01,
        lr_scheduler_type             = "linear",
        seed                          = 3407,
        output_dir                    = "outputs",
        report_to                     = "tensorboard",
        logging_dir                   = tensorboard_log_dir,
        logging_strategy              = "steps",
        save_strategy                 = "steps",
        save_steps                    = steps_per_epoch,
    ),
)

Change epochs, learning rate, batch size . max seq length etc according your data, while traning , moniter training loss and update your hyperparams according to that.

As example for 210k rows data, only example - taskydata/baize_chatbot

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import math

per_device_batch_size = 2 
grad_accum            = 16 
num_epochs            = 1  
max_steps = math.ceil((len(dataset["train"]) / (per_device_batch_size * grad_accum)) * num_epochs)

trainer = SFTTrainer(
    model            = model,
    tokenizer        = tokenizer,
    train_dataset    = dataset["train"],
    # eval_dataset   = dataset["test"], # Add this for evaluation
    # formatting_func= formatting_func,
    max_seq_length   = 2048, # Increased for chat data, adjust based on VRAM
    packing          = True, 
    args = TrainingArguments(
        per_device_train_batch_size = per_device_batch_size,
        gradient_accumulation_steps = grad_accum,
        warmup_ratio                = 0.05, 
        max_steps                   = max_steps,
        learning_rate               = 2e-5, 
        fp16                        = not is_bfloat16_supported(),
        bf16                        = is_bfloat16_supported(),
        logging_steps               = 25, # Log progress more reasonably
        optim                       = "adamw_8bit",
        weight_decay                = 0.01,
        lr_scheduler_type           = "cosine", 
        seed                        = 3407,
        output_dir                  = "outputs",
        report_to                   = "tensorboard",
        save_strategy               = "steps",
        save_steps                  = 500, # Save checkpoints periodically
        # evaluation_strategy       = "steps", # Enable evaluation
        # eval_steps                = 500,     # Evaluate every 500 steps
    ),
)